In [1]:
# Importação das bibliotecas a serem utilizadas durante todo o processo

import pandas as pd
import numpy as np
from datetime import datetime
from time import time
import math
import sys

In [2]:
start = time()
# Leitura do arquivo da tabela com as informações necessárias

df = pd.read_csv('Loja_44/loja_44_tabela_2_v12_1.csv')
df['COD_ITEM'] = df['COD_ITEM'].astype(str)
df['CHAVE_ITEM'] = df['CHV_DOC'].astype(str) + '-' + df['NUM_ITEM'].astype(str)

# Criação de dataset para checagem de duplicação de par CHAVE-NUM_ITEM
duplicate_mask = df['CHAVE_ITEM'].duplicated(keep=False)
duplicate_df = df[duplicate_mask]

# Checagem de Aliquota: TODOS os produtos devem ter Aliquota
if len(df[df['ALIQUOTA'].isnull()]) > 0:
    print('Existem registros sem aliquota. Favor checar o problema')
    sys.exit()

# Checagem de duplicidade: NÂO deve haver repetição de par CHV_DOC-NUM_ITEM
if len(duplicate_df) > 0:
    print('Existem combinações Chave-Numero que estão duplicadas. Favor checar o problema')
    sys.exit()
    
#Leitura do arquivo com as informações sobre o sub-tipo de operação por CFOP

df_2 = pd.read_excel('CFOP_e_template.xlsx', sheet_name='Tb_CFOP')

# Adição da informação do sub-tipo à tabela principal

df_new_1 = df.join(df_2.set_index('CFOP'), lsuffix='_orig', rsuffix='_externo', on='CFOP')
df_new_1['DATA'] = pd.to_datetime(df_new_1['DATA'], format='%Y-%m-%d')
df_new_1 = df_new_1.sort_values(by=['COD_ITEM', 'DATA', 'IND_OPER'])
df_new_1['COD_ITEM'] = df_new_1['COD_ITEM'].astype(str)

# Preenchimento da coluna VL_CONFR

vl_confr = []
for i, row in df_new_1.iterrows():
    if (row['IND_OPER'] == 1) and (row['SINAL'] == 1) and (row['CFOP'] != 5927):
        vl_confr.append(row['VALOR'] * (row['ALIQUOTA']/100))
    else:
        vl_confr.append(np.nan)
        
df_new_1['VL_CONFR'] = vl_confr
df_new_1['ICMS_TOT'] = df_new_1['ICMS_TOT'].astype(float).fillna(0)

# Criação de dataset para cálculo do ICMS inicial

data = df_new_1[['COD_ITEM', 'DATA', 'QTD_CAT', 'IND_OPER']]

# Transformação da coluna 'DATA' para o tipo correto
data['DATA'] = pd.to_datetime(data['DATA'], format='%Y-%m-%d')

# Ordenação do dataset para cálculo correto do ICMS inicial

data['IND_OPER'] = data['IND_OPER'].astype(int)
data = data.sort_values(by=['COD_ITEM','DATA', 'IND_OPER'])

# Cáculo dos saldos das operações para definição da Quantidade Inicial

saldos = []
for cod in data['COD_ITEM'].unique():
    qtd_ini = 0
    new = data[data['COD_ITEM'] == cod]
    for i,row in new.iterrows():

        if int(row['IND_OPER']) == 0:
            qtd_ini += int(row['QTD_CAT'])
            saldos.append(qtd_ini)
        else:
            qtd_ini -= int(row['QTD_CAT'])
            saldos.append(qtd_ini)
            
# Criação da coluna para as informações da Quantidade Inicial
df['QTD_INICIAL'] = [np.nan]* df.shape[0]

#Criação e população da coluna para as informações dos saldos
data['SALDOS'] = [np.nan]*data.shape[0]
data['SALDOS'] = saldos

# Criação da coluna para as informações da Quantidade Inicial

df['QTD_INICIAL'] = [np.nan]* df.shape[0]

# Cálculo das quantidades iniciais
qts_ini = []
for cod in data['COD_ITEM'].unique():
    min_ = data[data['COD_ITEM'] == cod]['SALDOS'].min()
    qts_ini.append(abs(min_))
    
# Criação de um novo dataset, com a informação de 'DATA' contendo somente mês e ano

new_data = data.copy()
mes_ano = []
for date in new_data['DATA']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])
    
new_data['DATA'] = mes_ano

# Listagem de todas as Datas(mes-ano) únicas
refs_unicas = list(new_data['DATA'].unique())

# Listagem de todos os códigos únicos
cods_unicos = list(new_data['COD_ITEM'].unique())

# Criação de um novo dataset para conter as informações finais sobre Quantidade inicial e ICMS inicial

new_df = pd.DataFrame(columns = ['REF', 'COD_ITEM', 'QTD_INI', 'ICMS_INI', 'QTD_FIM','ICMS_FIN'])

# Definição da lista de códigos únicos, sendo que cada código
# aparece uma vez para cada mês.
cod_item = cods_unicos*len(refs_unicas)

# Definição da lista de datas(mes-ano) únicas, sendo que cada data
# aparece uma quantidade de vezes igual a quantidade de códigos únicos

refs = refs_unicas*len(cods_unicos)

# Listagem e modificação das datas para posterior ordenação

refs_1 = []
for item in refs:
    refs_1.append(item[2:] + item[:2])
    
# Ordenação das datas
refs_ord = sorted(refs_1)

# Ajuste da formatação das datas ordenadas para o formato correto

refs_ord_final = []
for item in refs_ord:
    refs_ord_final.append(item[4:] + item[:4])
    
# Preenchimento das colunas dos Códigos dos itens
# e da coluna das datas(mes-ano)

new_df['COD_ITEM'] = cod_item
new_df['REF'] = refs_ord_final

# Cálculo das Quantidades iniciais

saldos = []
for cod in cods_unicos:
    saldo_min = data[data['COD_ITEM'] == cod]['SALDOS'].min()
    if saldo_min < 0:
        saldos.append(abs(saldo_min))
    else:
        saldos.append(0)
        
# Preenchimento da coluna das Quantidades iniciais
new_df['QTD_INI'] = saldos + [np.nan]*(len(cods_unicos)*(len(refs_unicas) -1))

new_df['REF'] = [datetime.strptime(x, '%m%Y').strftime('%Y%m') for x in new_df['REF']]
new_df = new_df.sort_values(by=['COD_ITEM', 'REF'])

# Preenchimento da coluna de Sub-tipo
df_new_1['SUB_TIPO'] = df_new_1['SINAL']

# Cálculo do ICMS inicial 

icms_init = []
data_novo = df_new_1.loc[(df_new_1['SUB_TIPO'] == 1) & (df_new_1['IND_OPER'] == 0)]
cods_unicos = list(data_novo['COD_ITEM'].unique())
icms_init_df = pd.DataFrame(columns = ['COD_ITEM', 'ICMS_INI'])
for cod in cods_unicos:
    
    sub = data_novo[data_novo['COD_ITEM'] == cod]
    if len(sub) > 5:
        sub_ = sub[:5]
        icmss_units = list(np.array(sub_['ICMS_TOT']) / np.array(sub_['QTD_CAT']))
        icms_init.append(max(icmss_units) * new_df[new_df['COD_ITEM'] == cod]['QTD_INI'].iloc[0])      
        
    else:
        sub_ = sub[0:]
        icmss_units = list(np.array(sub_['ICMS_TOT']) / np.array(sub_['QTD_CAT']))
        icms_init.append(max(icmss_units) * new_df[new_df['COD_ITEM'] == cod]['QTD_INI'].iloc[0])

for codigo in df_new_1['COD_ITEM'].unique():
    if codigo not in cods_unicos:
        cods_unicos.append(codigo)
        icms_init.append(0)


icms_init_df['COD_ITEM'] = cods_unicos
icms_init_df['ICMS_INI'] = icms_init        


# Criação de um dataset para conter as informações finais do 
# cálculo do ICMS inicial para o primeiro mês e preenchimento
# da informação do ICMS inicial

esqueleto_df = new_df[new_df['REF'] == '201910'].join(icms_init_df.set_index('COD_ITEM'), lsuffix='_orig', rsuffix='_externo', on='COD_ITEM')
 
# Definição contendo as informações para todos os meses
# além do mês inicial
df_resto = new_df[new_df['REF'] != '201910']
df_final = pd.concat([esqueleto_df, df_resto])

# Formatação e ajuste das colunas do dataset 
df_final = df_final[['REF', 'COD_ITEM', 'QTD_INI', 'ICMS_INI_externo', 'QTD_FIM', 'ICMS_FIN',]]
df_final.rename(columns={'ICMS_INI_externo': 'ICMS_INI'}, inplace=True)

tabela_2 = df_new_1[['DATA', 'CFOP', 'COD_ITEM', 'IND_OPER', 
                    'COD_LEGAL', 'SUB_TIPO']]

tabela_2['DATA'] = pd.to_datetime(tabela_2['DATA'], format='%d/%m/%Y')
tabela_2 = tabela_2.sort_values(by=['COD_ITEM', 'DATA', 'IND_OPER'])

tabela_2['SALDO_INI_QTD'] = [np.nan]*tabela_2.shape[0]
tabela_2['SALDO_INI_ICMS'] = [np.nan]*tabela_2.shape[0]
tabela_2['QTD_CAT'] = df_new_1['QTD_CAT']
tabela_2 = tabela_2[['DATA', 'CFOP', 'COD_ITEM', 'IND_OPER', 
                    'COD_LEGAL', 'SUB_TIPO',
                    'QTD_CAT','SALDO_INI_QTD','SALDO_INI_ICMS']]

df_final['COD_ITEM'] = df_final['COD_ITEM'].astype(str)
df_final_preen = df_final.sort_values(by=['COD_ITEM'])

df_final_preen['REF'] = df_final_preen['REF'].astype(str)
df_final_preen['REF'] = [datetime.strptime(x, '%Y%m').strftime('%m-%Y') for x in df_final_preen['REF']]

df_final_preen['REF'] = pd.to_datetime(df_final_preen['REF'], format='%m-%Y')
df_final_preen = df_final_preen.sort_values(by=['COD_ITEM', 'REF'])

saldo_inicial_qtd = [df_final_preen[df_final_preen['COD_ITEM'] == tabela_2['COD_ITEM'].iloc[0]]['QTD_INI'].iloc[0]]
for i in range(1, tabela_2.shape[0]):
    if tabela_2['COD_ITEM'].iloc[i] != tabela_2['COD_ITEM'].iloc[i-1]:
        saldo_inicial_qtd.append(df_final_preen[df_final_preen['COD_ITEM'] == tabela_2['COD_ITEM'].iloc[i]]['QTD_INI'].iloc[0])
                
    else:
        saldo_inicial_qtd.append(np.nan)
        
saldo_inicial_icms = [df_final_preen[df_final_preen['COD_ITEM'] == tabela_2['COD_ITEM'].iloc[0]]['ICMS_INI'].iloc[0]]
for i in range(1,tabela_2.shape[0]):
    if tabela_2['COD_ITEM'].iloc[i] != tabela_2['COD_ITEM'].iloc[i-1]:
        saldo_inicial_icms.append(df_final_preen[df_final_preen['COD_ITEM'] == tabela_2['COD_ITEM'].iloc[i]]['ICMS_INI'].iloc[0])
                
    else:
        saldo_inicial_icms.append(np.nan)
        
tabela_2['SALDO_INI_QTD'] = saldo_inicial_qtd
tabela_2['SALDO_INI_ICMS'] = saldo_inicial_icms

qtd_ent1_devolv_ent = []

for i, row in tabela_2.iterrows():
    if row['IND_OPER'] == 0:
        if row['SUB_TIPO'] == 1:
            qtd_ent1_devolv_ent.append(row['QTD_CAT'])
        else:
            qtd_ent1_devolv_ent.append(0)
    else:
        if row['SUB_TIPO'] == -1:
            qtd_ent1_devolv_ent.append(-row['QTD_CAT'])
        else:
            qtd_ent1_devolv_ent.append(0)
            
tabela_2['QTD_ent1_devolv_ent'] = qtd_ent1_devolv_ent

tabela_2['ICMS_TOT'] = df_new_1['ICMS_TOT']

icms_tot_ent_unit = []

for i, row in tabela_2.iterrows():
    if row['QTD_ent1_devolv_ent'] > 0:
        icms_tot_ent_unit.append(row['ICMS_TOT']/row['QTD_ent1_devolv_ent'])
    else:
        icms_tot_ent_unit.append(np.nan)
        
tabela_2['ICMS_TOT_ent_unit'] = icms_tot_ent_unit

mask = tabela_2['QTD_ent1_devolv_ent'] > 0

tabela_2['ICMS_TOT_ent_unit'] = np.where(mask, tabela_2['ICMS_TOT'] / tabela_2['QTD_ent1_devolv_ent'], 0)

ult_icms_tot_ent_unit = []

if tabela_2.iloc[0]['SALDO_INI_QTD'] == 0:
    ult_icms_tot_ent_unit.append(0.000001)
else:
    ult_icms_tot_ent_unit.append(tabela_2.iloc[0]['SALDO_INI_ICMS']/tabela_2.iloc[0]['SALDO_INI_QTD'])

for i in range(1, tabela_2.shape[0]):
    if tabela_2.iloc[i]['COD_ITEM'] != tabela_2.iloc[i-1]['COD_ITEM']:
        if tabela_2.iloc[i]['SALDO_INI_QTD'] == 0:
            ult_icms_tot_ent_unit.append(0.000001)
        else:
            ult_icms_tot_ent_unit.append(tabela_2.iloc[i]['SALDO_INI_ICMS']/tabela_2.iloc[i]['SALDO_INI_QTD'])    
    else:
        if tabela_2.iloc[i-1]['QTD_ent1_devolv_ent'] > 0:
            ult_icms_tot_ent_unit.append(tabela_2.iloc[i-1]['ICMS_TOT_ent_unit'])
        else:
            ult_icms_tot_ent_unit.append(ult_icms_tot_ent_unit[i-1])
    
tabela_2['ULT_ICMS_TOT_ent_unit'] = ult_icms_tot_ent_unit

mask = tabela_2['QTD_ent1_devolv_ent'] < 0

tabela_2['ICMS_TOT_1'] = np.where(mask,tabela_2['QTD_ent1_devolv_ent']*tabela_2['ULT_ICMS_TOT_ent_unit'],tabela_2['ICMS_TOT'])

qtd_saida_1_devolv_saida = []

for i, row in tabela_2.iterrows():
    if row['IND_OPER'] == 1:
        if row['SUB_TIPO'] == 1:
            qtd_saida_1_devolv_saida.append(row['QTD_CAT'])
        else:
            qtd_saida_1_devolv_saida.append(0)
    else:
        if row['SUB_TIPO'] == -1:
            qtd_saida_1_devolv_saida.append(-row['QTD_CAT'])
        else:
            qtd_saida_1_devolv_saida.append(0)

tabela_2['qtd_saida_1_devolv_saida'] = qtd_saida_1_devolv_saida
tabela_2['qtd_saida_1_devolv_saida'] = tabela_2['qtd_saida_1_devolv_saida'].astype(int)

tabela_2['COD_ITEM'] = tabela_2['COD_ITEM'].astype(str)
qtd_saldo = [] # AM
for code in tabela_2['COD_ITEM'].unique():
    qtd_saldo_int = []
    for i,row in tabela_2[tabela_2['COD_ITEM']==code].reset_index(drop=True).iterrows():
        # QTD_SALDO    
        if i == 0:
            qtd_saldo_int.append(row['SALDO_INI_QTD'] + row['QTD_ent1_devolv_ent'] - row['qtd_saida_1_devolv_saida'])
        else:
            qtd_saldo_int.append(qtd_saldo_int[i-1] + row['QTD_ent1_devolv_ent'] - row['qtd_saida_1_devolv_saida'])
        
    qtd_saldo += qtd_saldo_int


tabela_2['QTD_SALDO'] = qtd_saldo

# Definição do primeiro valor de ICMS_SAIDA_UNI
icms_saida_uni = [] # W

# Definição do primeiro valor de ULT_ICMS_SAIDA_UNI
ult_icms_saida_uni = [] # Y

# Definição do primeiro valor de ICMS_SAIDA
icms_saida = [] # AA
        
# Definição do primeiro valor de ICMS_TOT_SALDO
icms_tot_saldo = [] # AO
    
# Definição de todos os outros valores para cada campo acima
for code in tabela_2['COD_ITEM'].unique():
    
    icms_saida_uni_int = []
    ult_icms_saida_uni_int = []
    icms_saida_int = []
    icms_tot_saldo_int = []
    qtd_saida_1_devolv_saida = list(tabela_2[tabela_2['COD_ITEM']==code]['qtd_saida_1_devolv_saida'])
    qtd_saldo = list(tabela_2[tabela_2['COD_ITEM']==code]['QTD_SALDO'])
    
    
    for i,row in tabela_2[tabela_2['COD_ITEM']==code].reset_index(drop=True).iterrows():
        
    # ICMS_SAIDA_UNI
        
        if i == 0:
            if qtd_saida_1_devolv_saida[0] > 0:
                if row['SALDO_INI_QTD'] == 0:
                    icms_saida_uni_int.append(0)
                else:  
                    icms_saida_uni_int.append(row['SALDO_INI_ICMS'] / row['SALDO_INI_QTD'])
            else:
                icms_saida_uni_int.append(0)
        
        else:
            if (qtd_saida_1_devolv_saida[i] != 0) and (qtd_saldo[i-1] != 0):  
                icms_saida_uni_int.append(icms_tot_saldo_int[i-1] / qtd_saldo[i-1])               
            else:      
                icms_saida_uni_int.append(0)
      
    # ULT_ICMS_SAIDA_UNI
        if i== 0:
            if row['SALDO_INI_QTD'] == 0:
                ult_icms_saida_uni_int.append(0.000001)
            else:           
                ult_icms_saida_uni_int.append(
                    row['SALDO_INI_ICMS'] / row['SALDO_INI_QTD'] if row['SALDO_INI_QTD'] != 0 else float("nan"))
        else:
            if qtd_saida_1_devolv_saida[i-1] > 0:
                ult_icms_saida_uni_int.append(icms_saida_uni_int[i-1])
            else:
                ult_icms_saida_uni_int.append(ult_icms_saida_uni_int[i-1])
                
    
    # ICMS_SAIDA
        if qtd_saida_1_devolv_saida[i] < 0:
            icms_saida_int.append(qtd_saida_1_devolv_saida[i]*ult_icms_saida_uni_int[i])
        else:
            icms_saida_int.append(icms_saida_uni_int[i]*qtd_saida_1_devolv_saida[i])
    
        
    # ICMS_TOT_SALDO
        if i == 0:
            icms_tot_saldo_int.append(row['SALDO_INI_ICMS'] + row['ICMS_TOT_1'] - icms_saida_int[i])
        else:
            icms_tot_saldo_int.append(icms_tot_saldo_int[i-1] + row['ICMS_TOT_1'] - icms_saida_int[i])
    
    icms_saida_uni += icms_saida_uni_int
    ult_icms_saida_uni += ult_icms_saida_uni_int
    icms_saida += icms_saida_int
    icms_tot_saldo += icms_tot_saldo_int
    
tabela_2['ICMS_SAIDA_UNI'] = icms_saida_uni
tabela_2['ULT_ICMS_SAIDA_UNI'] = ult_icms_saida_uni
tabela_2['ICMS_SAIDA'] = icms_saida
tabela_2['ICMS_TOT_SALDO'] = icms_tot_saldo

icms_tot_pcat = []

for i, row in tabela_2.iterrows():
    if row['qtd_saida_1_devolv_saida'] < 0:
        icms_tot_pcat.append(np.abs(row['ICMS_SAIDA_UNI']))
    else:
        if row['QTD_ent1_devolv_ent'] != 0:
            icms_tot_pcat.append(np.abs(row['ICMS_TOT_1']))
        else:
            icms_tot_pcat.append(np.nan)
            
tabela_2['ICMS_TOT_PCAT'] = icms_tot_pcat

tabela_2['VLR_CONF_0'] = df_new_1['VL_CONFR']

mask = tabela_2['qtd_saida_1_devolv_saida'] > 0
vlr_confr_unit = np.where(mask, tabela_2['VLR_CONF_0']/tabela_2['qtd_saida_1_devolv_saida'], np.nan)

tabela_2['VLR_CONFR_UNIT'] = vlr_confr_unit

ult_vlr_confr_unit = [0]

for i in range(1, tabela_2.shape[0]):
    if tabela_2.iloc[i]['COD_ITEM'] != tabela_2.iloc[i-1]['COD_ITEM']:
        ult_vlr_confr_unit.append(0)
    else:
        if math.isnan(tabela_2.iloc[i-1]['VLR_CONFR_UNIT']):
            ult_vlr_confr_unit.append(ult_vlr_confr_unit[i-1])
        else:
            ult_vlr_confr_unit.append(tabela_2.iloc[i-1]['VLR_CONFR_UNIT'])
            
tabela_2['ULT_VLR_CONFR_UNIT'] = ult_vlr_confr_unit

vlr_confr_1 = []

for i, row in tabela_2.iterrows():
    if row['qtd_saida_1_devolv_saida'] > 0:
        vlr_confr_1.append(row['VLR_CONF_0'])
    else:
        if row['qtd_saida_1_devolv_saida'] < 0:
            vlr_confr_1.append(row['ULT_VLR_CONFR_UNIT'] * row['qtd_saida_1_devolv_saida'])
        else:
            vlr_confr_1.append(0)
            
tabela_2['VLR_CONFR_1'] = vlr_confr_1

mask = np.abs(tabela_2['ICMS_SAIDA']) > np.abs(tabela_2['VLR_CONFR_1'])
vlr_ressarc = np.where(mask, tabela_2['ICMS_SAIDA'] - tabela_2['VLR_CONFR_1'], 0)

tabela_2['VLR_RESSARCIMENTO'] = vlr_ressarc

mask = np.abs(tabela_2['ICMS_SAIDA']) < np.abs(tabela_2['VLR_CONFR_1'])
vlr_compl = np.where(mask, -tabela_2['ICMS_SAIDA'] + tabela_2['VLR_CONFR_1'], 0)

tabela_2['VLR_COMPLEMENTO'] = vlr_compl

cod_legal_pcat = []

for i, row in tabela_2.iterrows():
    if np.abs(row['VLR_RESSARCIMENTO']) > 0.1e-11:
        cod_legal_pcat.append(1)
    else:
        if row['VLR_COMPLEMENTO'] != 0:
            cod_legal_pcat.append(0)
        else:
            if row['qtd_saida_1_devolv_saida'] != 0:
                cod_legal_pcat.append(0)
            else:
                cod_legal_pcat.append(np.nan)
            
tabela_2['COD_LEGAL_PCAT'] = cod_legal_pcat

vlr_confr_pcat = []

for i, row in tabela_2.iterrows():
    if row['COD_LEGAL_PCAT'] == 1:
        vlr_confr_pcat.append(np.abs(row['VLR_CONFR_1']))
    else:
        if row['COD_LEGAL_PCAT'] == 0:
            vlr_confr_pcat.append(0)
        else:
            vlr_confr_pcat.append(np.nan)
            
tabela_2['VLR_CONFR_PCAT'] = vlr_confr_pcat

tabela_2['REF'] = tabela_2['DATA'].astype(str)
tabela_2['REF'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m-%Y') for x in tabela_2['REF']]

saldo_final_mes_qtd = []

for i in range(tabela_2.shape[0] - 1):
    if (tabela_2.iloc[i]['COD_ITEM'] != tabela_2.iloc[i+1]['COD_ITEM']) or (tabela_2.iloc[i]['REF'] != tabela_2.iloc[i+1]['REF']):
        saldo_final_mes_qtd.append(tabela_2.iloc[i]['QTD_SALDO'])
    else:
        saldo_final_mes_qtd.append(np.nan)

saldo_final_mes_qtd.append(tabela_2.iloc[-1]['QTD_SALDO'])


tabela_2['SALDO_FINAL_MES_QTD'] = saldo_final_mes_qtd

saldo_final_mes_icms = []
    
for i in range(tabela_2.shape[0] - 1):
    if (tabela_2.iloc[i]['COD_ITEM'] != tabela_2.iloc[i+1]['COD_ITEM']) or (tabela_2.iloc[i]['REF'] != tabela_2.iloc[i+1]['REF']):
        saldo_final_mes_icms.append(tabela_2.iloc[i]['ICMS_TOT_SALDO'])
    else:
        saldo_final_mes_icms.append(np.nan)

saldo_final_mes_icms.append(tabela_2.iloc[-1]['ICMS_TOT_SALDO'])

tabela_2['SALDO_FINAL_MES_ICMS'] = saldo_final_mes_icms



end = time()

print(end-start)

C:\Users\vinic\AppData\Local\Temp\ipykernel_14420\2370584205.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Loja_44/loja_44_tabela_2_v12_1.csv')
C:\Users\vinic\AppData\Local\Temp\ipykernel_14420\2370584205.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DATA'] = pd.to_datetime(data['DATA'], format='%Y-%m-%d')
C:\Users\vinic\AppData\Local\Temp\ipykernel_14420\2370584205.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

2180.8442227840424


In [3]:
df_final['REF'] = df_final['REF'].astype(str)
df_final['REF'] = [datetime.strptime(x, '%Y%m').strftime('%m-%Y') for x in df_final['REF']]

data_2 = tabela_2[['DATA','COD_ITEM','SALDO_FINAL_MES_QTD','SALDO_FINAL_MES_ICMS']]

data_2_novo = data_2.dropna()
data_2_novo['DATA'] = data_2_novo['DATA'].astype(str)
data_2_novo['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m') for x in data_2_novo['DATA']]

df_ord = df_final.copy()
df_ord['REF'] = df_ord['REF'].astype(str)
df_ord['REF'] = [datetime.strptime(x, '%m-%Y').strftime('%Y-%m') for x in df_ord['REF']]

df_ord = df_ord.sort_values(by=['COD_ITEM', 'REF'])

for i in range(data_2_novo.shape[0]):
    idx = df_ord.loc[(df_ord['REF'] == data_2_novo.iloc[i]['DATA']) & (df_ord['COD_ITEM'] == data_2_novo.iloc[i]['COD_ITEM'])].index[0]
    col_1 = 'QTD_FIM'
    col_2 = 'ICMS_FIN'
    valor_qtd = data_2_novo.loc[(data_2_novo['DATA'] == data_2_novo.iloc[i]['DATA']) & (data_2_novo['COD_ITEM']==data_2_novo.iloc[i]['COD_ITEM'])]['SALDO_FINAL_MES_QTD'].values[0]
    valor_icms = data_2_novo.loc[(data_2_novo['DATA'] == data_2_novo.iloc[i]['DATA']) & (data_2_novo['COD_ITEM']==data_2_novo.iloc[i]['COD_ITEM'])]['SALDO_FINAL_MES_ICMS'].values[0]
    df_ord.loc[idx, col_1] = valor_qtd
    df_ord.loc[idx, col_2] = valor_icms

    
qtd_inicial = []
qtd_fim = []

for code in df_ord['COD_ITEM'].unique():
    qtd_ini_int = []
    qtd_fin_int = []
    
    
    for i, row in df_ord[df_ord['COD_ITEM'] == code].reset_index().iterrows():
        
        if math.isnan(row['QTD_INI']):
            qtd_ini_int.append(qtd_fin_int[i-1])
        else:
            qtd_ini_int.append(row['QTD_INI'])
            
        if math.isnan(row['QTD_FIM']):
            qtd_fin_int.append(qtd_ini_int[i])
        else:
            qtd_fin_int.append(row['QTD_FIM'])
            
    qtd_inicial += qtd_ini_int
    qtd_fim += qtd_fin_int

    
icms_inicial = []
icms_fim = []

for code in df_ord['COD_ITEM'].unique():
    icms_ini_int = []
    icms_fin_int = []
    
    
    for i, row in df_ord[df_ord['COD_ITEM'] == code].reset_index().iterrows():
        
        if math.isnan(row['ICMS_INI']):
            try:
                icms_ini_int.append(icms_fin_int[i-1])
            except:
                icms_ini_int.append(0)
        else:
            icms_ini_int.append(row['ICMS_INI'])
            
        if math.isnan(row['ICMS_FIN']):
            try:
                icms_fin_int.append(icms_ini_int[i])
            except:
                icms_fin_int.append(0)
        else:
            icms_fin_int.append(row['ICMS_FIN'])
            
    icms_inicial += icms_ini_int
    icms_fim += icms_fin_int
    
df_ord['QTD_INI'] = qtd_inicial
df_ord['QTD_FIM'] = qtd_fim
df_ord['ICMS_INI'] = icms_inicial
df_ord['ICMS_FIN'] = icms_fim


C:\Users\vinic\AppData\Local\Temp\ipykernel_14420\2790365349.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2_novo['DATA'] = data_2_novo['DATA'].astype(str)
C:\Users\vinic\AppData\Local\Temp\ipykernel_14420\2790365349.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2_novo['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m') for x in data_2_novo['DATA']]


In [4]:
tabela_2['CHV_DOC'] = df_new_1['CHV_DOC']
tabela_2

,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,QTD_ent1_devolv_ent,...,ULT_VLR_CONFR_UNIT,VLR_CONFR_1,VLR_RESSARCIMENTO,VLR_COMPLEMENTO,COD_LEGAL_PCAT,VLR_CONFR_PCAT,REF,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,CHV_DOC
197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,0,...,0.0000,14.1030,0.0,2.278800,0.0,0.0,10-2019,NaN,NaN,35191071605265000595590003114260972468062852
1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,1,...,14.1030,0.0000,0.0,0.000000,NaN,NaN,10-2019,3.0,35.452800,35191005651966000455550240004004761539366521
305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,0,...,14.1030,14.1030,0.0,2.285400,0.0,0.0,11-2019,NaN,NaN,35191171605265000595590003114261029698537006
11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,1,...,14.1030,0.0000,0.0,0.000000,NaN,NaN,11-2019,NaN,NaN,35191127849963000110550010006031801006012824
316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,0,...,14.1030,14.1030,0.0,2.283200,0.0,0.0,11-2019,NaN,NaN,35191171605265000595590003114261058425299890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,1,...,15.1686,0.0000,0.0,0.000000,NaN,NaN,03-2020,2.0,27.292031,35200345453214001042550180007353931100320204
247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,0,...,15.1686,15.1686,0.0,1.522584,0.0,0.0,05-2020,1.0,13.646016,35200571605265000595590003114261366257626039
350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,0,...,15.1686,15.1686,0.0,1.522584,0.0,0.0,06-2020,0.0,0.000000,35200671605265000595590003114261397752679870
89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,2,...,15.1686,0.0000,0.0,0.000000,NaN,NaN,07-2020,2.0,27.240000,35200744463156000184550840006132201109539626


In [5]:
tabela_2['NUM_ITEM'] = df_new_1['NUM_ITEM']
tabela_2

,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,QTD_ent1_devolv_ent,...,VLR_CONFR_1,VLR_RESSARCIMENTO,VLR_COMPLEMENTO,COD_LEGAL_PCAT,VLR_CONFR_PCAT,REF,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,CHV_DOC,NUM_ITEM
197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,0,...,14.1030,0.0,2.278800,0.0,0.0,10-2019,NaN,NaN,35191071605265000595590003114260972468062852,1
1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,1,...,0.0000,0.0,0.000000,NaN,NaN,10-2019,3.0,35.452800,35191005651966000455550240004004761539366521,8
305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,0,...,14.1030,0.0,2.285400,0.0,0.0,11-2019,NaN,NaN,35191171605265000595590003114261029698537006,1
11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,1,...,0.0000,0.0,0.000000,NaN,NaN,11-2019,NaN,NaN,35191127849963000110550010006031801006012824,17
316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,0,...,14.1030,0.0,2.283200,0.0,0.0,11-2019,NaN,NaN,35191171605265000595590003114261058425299890,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,1,...,0.0000,0.0,0.000000,NaN,NaN,03-2020,2.0,27.292031,35200345453214001042550180007353931100320204,2
247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,0,...,15.1686,0.0,1.522584,0.0,0.0,05-2020,1.0,13.646016,35200571605265000595590003114261366257626039,1
350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,0,...,15.1686,0.0,1.522584,0.0,0.0,06-2020,0.0,0.000000,35200671605265000595590003114261397752679870,1
89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,2,...,0.0000,0.0,0.000000,NaN,NaN,07-2020,2.0,27.240000,35200744463156000184550840006132201109539626,4


In [6]:
tabela_2['CNPJ EMITENTE'] = df_new_1['CNPJ EMITENTE']
tabela_2

,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,QTD_ent1_devolv_ent,...,VLR_RESSARCIMENTO,VLR_COMPLEMENTO,COD_LEGAL_PCAT,VLR_CONFR_PCAT,REF,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,CHV_DOC,NUM_ITEM,CNPJ EMITENTE
197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,0,...,0.0,2.278800,0.0,0.0,10-2019,NaN,NaN,35191071605265000595590003114260972468062852,1,NaN
1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,1,...,0.0,0.000000,NaN,NaN,10-2019,3.0,35.452800,35191005651966000455550240004004761539366521,8,5.651966e+12
305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,0,...,0.0,2.285400,0.0,0.0,11-2019,NaN,NaN,35191171605265000595590003114261029698537006,1,NaN
11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,1,...,0.0,0.000000,NaN,NaN,11-2019,NaN,NaN,35191127849963000110550010006031801006012824,17,2.784996e+13
316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,0,...,0.0,2.283200,0.0,0.0,11-2019,NaN,NaN,35191171605265000595590003114261058425299890,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,1,...,0.0,0.000000,NaN,NaN,03-2020,2.0,27.292031,35200345453214001042550180007353931100320204,2,4.545321e+13
247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,0,...,0.0,1.522584,0.0,0.0,05-2020,1.0,13.646016,35200571605265000595590003114261366257626039,1,NaN
350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,0,...,0.0,1.522584,0.0,0.0,06-2020,0.0,0.000000,35200671605265000595590003114261397752679870,1,NaN
89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,2,...,0.0,0.000000,NaN,NaN,07-2020,2.0,27.240000,35200744463156000184550840006132201109539626,4,4.446316e+13


In [7]:
tabela_2['DESCRICAO'] = df_new_1['DESCRICAO']
tabela_2['CODIGO_BARRA'] = df_new_1['CODIGO_BARRA']
tabela_2['UNIDADE'] = df_new_1['UNIDADE']
tabela_2['N C M'] = df_new_1['N C M']
tabela_2['ALIQUOTA'] = df_new_1['ALIQUOTA']
tabela_2['CEST'] = df_new_1['CEST']

In [8]:
tabela_2_t = tabela_2[['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM', 'IND_OPER', 'SUB_TIPO',
                    'QTD_CAT', 'SALDO_INI_QTD', 'SALDO_INI_ICMS']]
tabela_2_t

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS
197762,35191071605265000595590003114260972468062852,2019-10-01,5405,1,100073,1,1,1,3.0,35.4726
1854,35191005651966000455550240004004761539366521,2019-10-03,1403,8,100073,0,1,1,NaN,NaN
305534,35191171605265000595590003114261029698537006,2019-11-01,5405,1,100073,1,1,1,NaN,NaN
11949,35191127849963000110550010006031801006012824,2019-11-06,1403,17,100073,0,1,1,NaN,NaN
316600,35191171605265000595590003114261058425299890,2019-11-21,5405,1,100073,1,1,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
54999,35200345453214001042550180007353931100320204,2020-03-11,1403,2,99671,0,1,1,NaN,NaN
247963,35200571605265000595590003114261366257626039,2020-05-25,5405,1,99671,1,1,1,NaN,NaN
350612,35200671605265000595590003114261397752679870,2020-06-13,5405,1,99671,1,1,1,NaN,NaN
89471,35200744463156000184550840006132201109539626,2020-07-15,1403,4,99671,0,1,2,NaN,NaN


In [9]:
tabela_2_1 = tabela_2[['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM', 'IND_OPER', 'SUB_TIPO',
                    'QTD_CAT', 'SALDO_INI_QTD', 'SALDO_INI_ICMS', 'QTD_ent1_devolv_ent',
                    'ICMS_TOT', 'ICMS_TOT_ent_unit', 'ULT_ICMS_TOT_ent_unit', 'ICMS_TOT_1',
                    'qtd_saida_1_devolv_saida', 'ICMS_SAIDA_UNI', 'ULT_ICMS_SAIDA_UNI',
                    'ICMS_SAIDA', 'ICMS_TOT_PCAT', 'VLR_CONF_0', 'VLR_CONFR_UNIT', 
                    'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'QTD_SALDO', 'ICMS_TOT_SALDO', 
                    'VLR_RESSARCIMENTO', 'VLR_COMPLEMENTO', 'COD_LEGAL', 'COD_LEGAL_PCAT',
                    'VLR_CONFR_PCAT', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
tabela_2_1

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,VLR_CONFR_1,QTD_SALDO,ICMS_TOT_SALDO,VLR_RESSARCIMENTO,VLR_COMPLEMENTO,COD_LEGAL,COD_LEGAL_PCAT,VLR_CONFR_PCAT,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS
197762,35191071605265000595590003114260972468062852,2019-10-01,5405,1,100073,1,1,1,3.0,35.4726,...,14.1030,2.0,23.648400,0.0,2.278800,1.0,0.0,0.0,NaN,NaN
1854,35191005651966000455550240004004761539366521,2019-10-03,1403,8,100073,0,1,1,NaN,NaN,...,0.0000,3.0,35.452800,0.0,0.000000,NaN,NaN,NaN,3.0,35.452800
305534,35191171605265000595590003114261029698537006,2019-11-01,5405,1,100073,1,1,1,NaN,NaN,...,14.1030,2.0,23.635200,0.0,2.285400,1.0,0.0,0.0,NaN,NaN
11949,35191127849963000110550010006031801006012824,2019-11-06,1403,17,100073,0,1,1,NaN,NaN,...,0.0000,3.0,35.459400,0.0,0.000000,NaN,NaN,NaN,NaN,NaN
316600,35191171605265000595590003114261058425299890,2019-11-21,5405,1,100073,1,1,1,NaN,NaN,...,14.1030,2.0,23.639600,0.0,2.283200,1.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54999,35200345453214001042550180007353931100320204,2020-03-11,1403,2,99671,0,1,1,NaN,NaN,...,0.0000,2.0,27.292031,0.0,0.000000,NaN,NaN,NaN,2.0,27.292031
247963,35200571605265000595590003114261366257626039,2020-05-25,5405,1,99671,1,1,1,NaN,NaN,...,15.1686,1.0,13.646016,0.0,1.522584,1.0,0.0,0.0,1.0,13.646016
350612,35200671605265000595590003114261397752679870,2020-06-13,5405,1,99671,1,1,1,NaN,NaN,...,15.1686,0.0,0.000000,0.0,1.522584,1.0,0.0,0.0,0.0,0.000000
89471,35200744463156000184550840006132201109539626,2020-07-15,1403,4,99671,0,1,2,NaN,NaN,...,0.0000,2.0,27.240000,0.0,0.000000,NaN,NaN,NaN,2.0,27.240000


In [10]:
df_ord.to_excel('Loja_44/1050_loja_44_version_final_v7.xlsx')

In [11]:
df_ord.to_csv('Loja_44/1050_loja_44_final_v7.csv')

In [12]:
tabela_2.to_csv('Loja_44/tabela_2_loja_44_version_final_v9.csv')

In [13]:
tabela_2_1.to_csv('Loja_44/tabela_2_loja_44_vers_v11.csv')

In [14]:
tabela_2_1.to_excel('Loja_44/tabela_2_loja_44_vers_v11.xlsx')

In [15]:
tabela_2.to_excel('Loja_44/tabela_2_loja_44_version_final_v9.xlsx')